# AttackRecipe
`AttackRecipe` aims to find a perturbation of an input text satisfies the attack's goal to fool the given `FlintModel`. In contrast to `Transformation`, `AttackRecipe` requires the prediction scores of the target model.  textflint provides an interface to integrate the easy-to-use adversarial attack recipes implemented based on `textattack`. Users can refer to [textattack](https://github.com/QData/TextAttack) for more information about the supported `AttackRecipe`. This section provides a brief introduction to how to use `AttackRecipe` in textflint.



In [1]:
import sys
sys.path.append('/home/yjc/codes/textrobustness')

## Using an `AttackRecipe` based on `textattack`

1. Define a list of `AttackRecipe` in a python file without defining the specific victim model. For example, we create a `attack_ins.py` file with the following commands:

```python
from textattack.goal_functions import UntargetedClassification
from textattack.search_methods import GreedySearch
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.transformations import WordSwapWordNet
from textflint.generation_layer.attack import Attack # Note that here we use the Attack from textflint

# Define the goal function class
goal_function = UntargetedClassification
# We'll constrain modification of already modified indices and stopwords
constraints = [RepeatModification(),
               StopwordModification()]
# We're going to use WordSwapWordNet as the attack transformation.
transformation = WordSwapWordNet()
# We'll use the Greedy search method
search_method = GreedySearch()
# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

# ... 
# many attacks form an attack list
attacks = [attack]
```

2. Define the path of above file in the config json file. For example, the config file `SA.json` might look as follows:

```json
{
  "task": "SA",
  "max_trans": 1,
  "semantic_validate": false,
  "semantic_score": 0.7,
  "fields": "x",
  "keep_origin": false,
  "return_unk": true,
  "transformation_config": {},
  "transformation_methods": [],
  "subpopulation_methods": [],
  "attack_methods": "attack_ins.py" //path to attack_ins.py
}
```

3. Load the SA test dataset:

In [2]:
from textflint.input_layer.model.test_model.model_helper import data_loader_csv
sa_data_set = data_loader_csv('/home/yjc/codes/textrobustness/textflint/input_layer/model/test_model/train.csv')
test_data_set = sa_data_set[int(len(sa_data_set) * 0.7):]

4. Create your own modelwrapper that implementing the function `evaluate` and `encode`. More details can be found in the `modelwrapper` tutorial.

In [3]:
from textflint.input_layer.model.wrappers.textcnn_torch_wrapper import TextCNNTorchWrapper
textcnn_wrapper = TextCNNTorchWrapper()

5. Feeding the dataset `test_data_set`, output path `out_dir_path`, config file `config` and model `textcnn_wrapper` to the SA engine, and run it! textflint will automatically scan the `attack_ins.py` file and load the `attacks` inside. 

In [4]:
from textflint.engine import Engine
from textflint.input_layer.config.config import Config

config = Config.from_json_file('/home/yjc/codes/textrobustness/textflint/common/config_files/SA/SA.json')
out_dir_path = '/home/yjc/codes/textrobustness/test_result/'

engine = Engine('SA')
engine.run(test_data_set, out_dir_path, config, textcnn_wrapper)

textflint: ******Start load!******
100%|██████████| 1544/1544 [00:00<00:00, 2156.11it/s]
textflint: 1544 in total, 1544 were loaded successful.
textflint: ******Finish load!******
textflint: ******Start DoubleDenial!******
100%|██████████| 1544/1544 [00:00<00:00, 6363.64it/s]
textflint: DoubleDenial, original 1544 samples, transform 281 samples!
textflint: Save samples to /home/yjc/codes/textrobustness/test_result/ori_DoubleDenial_281.json!
textflint: Save samples to /home/yjc/codes/textrobustness/test_result/trans_DoubleDenial_281.json!
textflint: ******Finish DoubleDenial!******


The adverisial samples based on the `AttackRecipe` will be also automatically saved to the directory `out_dir_path`, and we can take a quick look at the contents:

In [5]:
with open('/home/yjc/codes/textrobustness/test_result/ori_DoubleDenial_267.json', 'r') as f:
    for ex in f.readlines()[:2]:
        print("original: ", ex)
        
with open('/home/yjc/codes/textrobustness/test_result/trans_DoubleDenial_267.json', 'r') as f:
    for ex in f.readlines()[:2]:
        print("transformed: ", ex)

original:  {"x": " @user \"#love is not  , but brings only sadness, because it can not be hold.\" erich maria remarque  mountain valley ", "y": "negative", "sample_id": 10}

original:  {"x": "#bustyescos  haha really great! @user @user @user ", "y": "positive", "sample_id": 19}

transformed:  {"x": "@user \" # don't hate is not, but brings only sadness, because it cannot be hold. \" erich maria remarque mountain valley", "y": "negative", "sample_id": 10}

transformed:  {"x": "# bustyescos haha really not bad! @user @user @user", "y": "positive", "sample_id": 19}



## Conclusion
In this tutorial, we briefly describe how to use `textattack`'s `AttackRecipe` to generate adverisial samples. We also support loading multiple attacks at once and executing them all by simply runing the `engine`. 